# Notebook to update list of functions in README.md

In [3]:
import re

import project

import pandas as pd

from inspect import Signature

from mintalib import funcs

def get_info(func):
    """ information about function """

    info = dict(Name=func.__qualname__)

    params = list(Signature.from_callable(func).parameters.values())
    if params and params[0].name in ('series', 'prices'):
        info.update(Input=params[0].name)

    doc = func.__doc__ or ""
    description = doc.strip().partition("\n")[0]
    if description is not None:
        info.update(Description=description)

    return info


def list_functions():
    """ list functions """

    result = [v for k, v in vars(funcs).items()
              if k.isupper() and callable(v)]

    result = [get_info(f) for f in result]

    result = pd.DataFrame(result).set_index('Name')

    return result


In [4]:
functions = list_functions()
print(functions.to_markdown())


| Name             | Input   | Description                                               |
|:-----------------|:--------|:----------------------------------------------------------|
| AVGPRICE         | prices  | Average Price                                             |
| TYPPRICE         | prices  | Typical Price                                             |
| WCLPRICE         | prices  | Weighted Close Price                                      |
| MIDPRICE         | prices  | Mid Price                                                 |
| PRICE            | prices  | Generic Price                                             |
| CROSSOVER        | series  | 1 when data cross over level, 0.0 elsewhere               |
| CROSSUNDER       | series  | 1 when data cross under level, 0.0 elsewhere              |
| FLAG_ABOVE       | series  | returns flag for strictly positive values                 |
| FLAG_BELOW       | series  | returns flag for strictly negative values                 |

In [3]:
def backup_readme(verbose=True):
    readme = project.root.joinpath("README.md")
    backup = project.root.joinpath("README.bak")

    if verbose:
        print(f"Backing up {readme.name} ...")

    contents = readme.read_text()
    size = backup.write_text(contents)
    return size

backup_readme()


Backing up README.md ...


3157

In [4]:

def update_readme(verbose=True):
    title = "## List of Functions\n"
    table = list_functions().to_markdown()
    repl = title + "\n" + table + "\n"

    # flags m=multiline, s=dotall
    pattern = r"(?ms)(^[#]+ List of (Functions|Indicators).*\Z)"

    readme = project.root.joinpath("README.md")
    contents = readme.read_text()

    output, count = re.subn(pattern, repl, contents)

    if count != 1:
        raise RuntimeError("Cound not locate list of funtions")

    if verbose:
        print(f"Updating {readme.name} ...")

    return readme.write_text(output)



update_readme()



Updating README.md ...


8072